# Housing Price Prediction with Amazon SageMaker Autopilot
_**Using Autopilot to Predict House Prices in California**_


Kernel `Python 3 (Data Science)` works well with this notebook within SageMaker Studio. If you open it in Notebook Instance, `conda_py3` is sufficient. 

---

## Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Prepare Training Data](#Data)
1. [Train](#Settingup)
1. [Autopilot Results](#Results)
1. [Evaluate Using Test Data](#Evaluate)
1. [Cleanup](#Cleanup)


---

## Introduction

Amazon SageMaker Autopilot is an automated machine learning (commonly referred to as AutoML) solution for tabular datasets. You can use SageMaker Autopilot in different ways: on autopilot (without any human input) or with human guidance, without code through SageMaker Studio or scripted using the AWS SDKs. This notebook will use the AWS SDKs to simply create and deploy a machine learning model without doing any feature engineering manually. We will also explore the auto-generated feature importance report.

Predicting housing prices is a classic linear regression problem in ML. The data pertains to the houses found in a given California district (as a group) and include summary stats about them based on census data from 1990. The dataset variables are easily understandable, and the columns are as follows:

* ```longitude```
* ```latitude```
* ```housingMedianAge```
* ```totalRooms```
* ```totalBedrooms```
* ```population```
* ```households```
* ```medianIncome```
* ```medianHouseValue``` (target)

What we're going to try to predict is the median house value for a district. We will let Autopilot perform feature engineering, model selection, model tuning, and give us the best candidate model ready to use for inferences.

---
## Setup

_This notebook was created and tested on a ml.t3.medium notebook instance._

Let's start by specifying:

- The S3 bucket and prefix to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting. The following code will use SageMaker's default S3 bucket (and create one if it doesn't exist).
- The IAM role ARN used to give training and hosting access to your data. See the documentation for how to create these. The following code will use the SageMaker execution role.

In [2]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()

# You can modify the following to use a bucket of your choosing
bucket = session.default_bucket()
prefix = "sagemaker/DEMO-autopilot-housing"

role = get_execution_role()

# This is the client we will use to interact with SageMaker Autopilot
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

Next, we'll import the Python libraries we'll need for the remainder of the exercise.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from urllib.parse import urlparse
import time

---
## Prepare Training Data<a name="Data"></a>

We will use the publicly available California housing dataset. The target variable is the median house value for California districts.

This dataset was obtained from the StatLib repository (http://lib.stat.cmu.edu/datasets/) and derived from the 1990 U.S. census, using one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).

In [4]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/california_housing/cal_housing.tgz .

download: s3://sagemaker-sample-files/datasets/tabular/california_housing/cal_housing.tgz to ./cal_housing.tgz


In [5]:
!tar -zxf cal_housing.tgz -o

In [6]:
columns = [
    "longitude",
    "latitude",
    "housingMedianAge",
    "totalRooms",
    "totalBedrooms",
    "population",
    "households",
    "medianIncome",
    "medianHouseValue",
]

target = "medianHouseValue"

cal_housing_df = pd.read_csv("CaliforniaHousing/cal_housing.data", names=columns, header=None)

Before starting an Autopilot job, it is good practice to inspect the data to ensure there are no obvious issues with it. 

In [7]:
pd.set_option("display.max_columns", 500)
cal_housing_df

,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


Notice that the various columns have values in different ranges. For example, values under the target column ```medianHouseValue``` are orders of magnitude higher than other columns. This difference in scale often causes issues when training an ML model, which is why it's a common feature engineering practice to normalize or standardize values (depending on the nature of the numeric distribution and presence of outliers).

However, because Autopilot handles feature engineering automatically (among other things), we're not going to make any analysis or transformations ourselves.

To also illustrate how Autopilot handles data issues, such as missing values, let's introduce some random empty values in our dataset for the housing median age column.


In [8]:
# Take a small, random sample of the dataset
cal_housing_df_sample = cal_housing_df.sample(frac=0.01, random_state=100)

cal_housing_df["housingMedianAge"].loc[cal_housing_df_sample.index] = (
    cal_housing_df["housingMedianAge"].loc[cal_housing_df_sample.index].apply(lambda row: np.nan)
)

Now our dataset should contain empty values for ~1% of the rows under the ```housingMedianAge``` column.

In [9]:
isna_sum = cal_housing_df["housingMedianAge"].isna().sum()
print("{0} values removed from housingMedianAge column".format(isna_sum))

206 values removed from housingMedianAge column


In [10]:
# Split data for training and testing (80/20 split)

train_data = cal_housing_df.sample(frac=0.8, random_state=200)

test_data = cal_housing_df.drop(train_data.index)

test_data_no_target = test_data.drop(columns=[target])

In [11]:
### Upload the dataset to S3

train_file = "train_data.csv"
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print("Train data uploaded to: " + train_data_s3_path)

test_file = "test_data_no_target.csv"
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print("Test data uploaded to: " + test_data_s3_path)

Train data uploaded to: s3://sagemaker-us-west-2-060090418572/sagemaker/DEMO-autopilot-housing/train/train_data.csv
Test data uploaded to: s3://sagemaker-us-west-2-060090418572/sagemaker/DEMO-autopilot-housing/test/test_data_no_target.csv


---
## Setting up the SageMaker Autopilot Job<a name="Settingup"></a>

After uploading the dataset to Amazon S3, you can invoke Autopilot to find the best ML pipeline to train a model on this dataset. 

The required inputs for invoking an Autopilot job are:
* Amazon S3 location for input dataset and for all output artifacts
* Name of the target column in the dataset for predictions 
* An IAM role

Currently, Autopilot supports only tabular datasets in CSV format. Either all files should have a header row, or the first file of the dataset, when sorted in alphabetical/lexical order by name, is expected to have a header row.


In [12]:
input_data_config = [
    {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": "s3://{}/{}/train".format(bucket, prefix),
            }
        },
        "TargetAttributeName": target,
    }
]

job_config = {"CompletionCriteria": {"MaxCandidates": 10}}


output_data_config = {"S3OutputPath": "s3://{}/{}/output".format(bucket, prefix)}

You can also specify the type of problem you want to solve with your dataset (`Regression, MulticlassClassification, BinaryClassification`). In case you are not sure, SageMaker Autopilot will infer the problem type based on statistics of the target column (the column you want to predict). 

Because the target attribute, ```medianHouseValue```, is a continuous numeric variable, Autopilot will infer the problem type as regression. 

You have the option to limit the running time of a SageMaker Autopilot job by providing either the maximum number of pipeline evaluations or candidates (one pipeline evaluation is called a `Candidate` because it generates a candidate model) or providing the total time allocated for the overall Autopilot job. Under default settings, this job may take several hours to run. This varies between runs because of the nature of the exploratory process Autopilot uses to find optimal training parameters.

For this demo, we limit the number of candidates to 10 so that the job finishes in under 1 hour.

Finally, you also have the option to deploy the winning model to a SageMaker endpoint automatically upon completion. In this case, we will not deploy the endpoint. We'll run a batch prediction job later instead to evaluate our model.

For guidance on how to configure the job parameters, check out the SDK documentation.

### Launching the SageMaker Autopilot Job<a name="Launching"></a>

You can now launch the Autopilot job by calling the `create_auto_ml_job` API. 

In [13]:
from time import gmtime, strftime, sleep

timestamp_suffix = strftime("%Y%m%d-%H-%M", gmtime())

auto_ml_job_name = "automl-housing-" + timestamp_suffix
print("AutoMLJobName: " + auto_ml_job_name)

sm.create_auto_ml_job(
    AutoMLJobName=auto_ml_job_name,
    InputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLJobConfig=job_config,
    # Uncomment to automatically deploy an endpoint
    # ModelDeployConfig={
    #'AutoGenerateEndpointName': True,
    #'EndpointName': 'autopilot-DEMO-housing-' + timestamp_suffix
    # },
    RoleArn=role,
)

AutoMLJobName: automl-housing-20220812-16-18


{'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:060090418572:automl-job/automl-housing-20220812-16-18',
 'ResponseMetadata': {'RequestId': '3c64c921-a0c9-419e-8793-a768c687719d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3c64c921-a0c9-419e-8793-a768c687719d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Fri, 12 Aug 2022 16:18:41 GMT'},
  'RetryAttempts': 0}}

---
The Autopilot job will now be performing the following steps:

* Data Analysis
* Feature Engineering
* Model selection
* Model tuning (hyperparameter optimization)
* Model feature importance (SageMaker Clarify)


### Tracking SageMaker Autopilot job progress

In [14]:
print("JobStatus - Secondary Status")
print("------------------------------")


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"])
job_run_status = describe_response["AutoMLJobStatus"]

while job_run_status not in ("Failed", "Completed", "Stopped"):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response["AutoMLJobStatus"]

    print(
        describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"]
    )
    sleep(60)

JobStatus - Secondary Status
------------------------------
InProgress - Starting
InProgress - Starting
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - ModelTuning
InProgress - GeneratingExplainabilityReport
InProgress - GeneratingExplainabilityReport
InProgress - GeneratingExplainabilityReport
InProgress - GeneratingExplainabilityReport
InProgress - GeneratingExplainabil

---
## Results

Now you can use the ```describe_auto_ml_job``` API to look up the best candidate selected by the SageMaker Autopilot job. 

In [15]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)["BestCandidate"]
best_candidate_name = best_candidate["CandidateName"]

print("\n")
print("CandidateName: " + best_candidate_name)
print(
    "FinalAutoMLJobObjectiveMetricName: "
    + best_candidate["FinalAutoMLJobObjectiveMetric"]["MetricName"]
)
print(
    "FinalAutoMLJobObjectiveMetricValue: "
    + str(best_candidate["FinalAutoMLJobObjectiveMetric"]["Value"])
)
print("\nBest candidate details:: " + str(best_candidate))



CandidateName: automl-housing-20220812-16-18wm1-010-352d0c11
FinalAutoMLJobObjectiveMetricName: validation:mse
FinalAutoMLJobObjectiveMetricValue: 2269658112.0

Best candidate details:: {'CandidateName': 'automl-housing-20220812-16-18wm1-010-352d0c11', 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:mse', 'Value': 2269658112.0}, 'ObjectiveStatus': 'Succeeded', 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:060090418572:processing-job/automl-housing-20220812-16-18-db-1-b51a80c4215742a88aaec179b3f7', 'CandidateStepName': 'automl-housing-20220812-16-18-db-1-b51a80c4215742a88aaec179b3f7'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:060090418572:training-job/automl-housing-20220812-16-18-dpp6-1-57bab6a84ec841918f9a3ec7c1', 'CandidateStepName': 'automl-housing-20220812-16-18-dpp6-1-57bab6a84ec841918f9a3ec7c1'}, {'CandidateStepType': 'AWS::SageMa

If you are curious to explore the performance of other algorithms that Autopilot explored, you can enumerate them via ```list_candidates_for_auto_ml_job``` API call

In [16]:
sm_dict = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name)
for item in sm_dict["Candidates"]:
    print(item["CandidateName"], item["FinalAutoMLJobObjectiveMetric"])
    print(item["InferenceContainers"][1]["Image"], "\n")

automl-housing-20220812-16-18wm1-007-ef23913c {'MetricName': 'validation:mse', 'Value': 2503931904.0}
246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3 

automl-housing-20220812-16-18wm1-010-352d0c11 {'MetricName': 'validation:mse', 'Value': 2269658112.0}
246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3 

automl-housing-20220812-16-18wm1-009-a4b9b267 {'MetricName': 'validation:mse', 'Value': 2280074240.0}
246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3 

automl-housing-20220812-16-18wm1-008-eeb25178 {'MetricName': 'validation:mse', 'Value': 3893984000.0}
246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3 

automl-housing-20220812-16-18wm1-006-e5194d10 {'MetricName': 'validation:mse', 'Value': 2470908672.0}
246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3 

automl-housing-20220812-16-18wm1-005-f2aa18f0 {'MetricName': 'validation:mse', 'Value': 2520620

---
Autopilot automatically generates two executable Jupyter Notebooks:  

- ```SageMakerAutopilotDataExplorationNotebook.ipynb```
- ```SageMakerAutopilotCandidateDefinitionNotebook.ipynb```

These notebooks are stored in S3. Let us download them onto our SageMaker Notebook instance, so we can explore them.

In [17]:
candidate_nbk = describe_response["AutoMLJobArtifacts"]["CandidateDefinitionNotebookLocation"]
data_explore_nbk = describe_response["AutoMLJobArtifacts"]["DataExplorationNotebookLocation"]


def split_s3_path(s3_path):
    path_parts = s3_path.replace("s3://", "").split("/")
    bucket = path_parts.pop(0)
    key = "/".join(path_parts)
    return bucket, key


s3_bucket, candidate_nbk_key = split_s3_path(candidate_nbk)
_, data_explore_nbk_key = split_s3_path(data_explore_nbk)

print(s3_bucket, candidate_nbk_key, data_explore_nbk_key)

session.download_data(path="./", bucket=s3_bucket, key_prefix=candidate_nbk_key)

session.download_data(path="./", bucket=s3_bucket, key_prefix=data_explore_nbk_key)

sagemaker-us-west-2-060090418572 sagemaker/DEMO-autopilot-housing/output/automl-housing-20220812-16-18/sagemaker-automl-candidates/automl-housing-20220812-16-18-pr-1-28caef255c6a4f80845cd1a3c702/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb sagemaker/DEMO-autopilot-housing/output/automl-housing-20220812-16-18/sagemaker-automl-candidates/automl-housing-20220812-16-18-pr-1-28caef255c6a4f80845cd1a3c702/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb


#### Data Exploration and Candidate Generation Notebooks

**Take some time to inspect the Data Exploration notebook. Check the ```Column Analysis and Descriptive Statistics``` section to see the analysis carried out by Autopilot.**

**Now, take some time to inspect the Candidate Generation notebook. Check the ```Generated Candidates``` section to see the different algorithms and data transformation strategies used by Autopilot.**

---
Autopilot also automatically generates a feature importance report.


In [18]:
explainability_prefix = best_candidate["CandidateProperties"]["CandidateArtifactLocations"][
    "Explainability"
]

s3_bucket, explainability_dir = split_s3_path(explainability_prefix)

session.download_data(path="./", bucket=s3_bucket, key_prefix=explainability_dir)

The preceding code will download a directory to our local environment. In that directory (the prefix is the autopilot job name, the suffix is automatically generated), you should see the SageMaker Clarify artifacts. SageMaker Clarify provides greater visibility into training data and models to identify and limit bias and explain predictions. The following code will open the feature importance report:

In [19]:
from IPython.display import IFrame

# Fetch the auto-generated directory name for the SageMaker Clarify artifacts
dir_name = (
    session.list_s3_files(bucket=s3_bucket, key_prefix=explainability_dir)[0]
    .replace(explainability_dir, "")
    .split("/")[1]
)

# Display HTML report
IFrame(src=f"{dir_name}/report.html", width=700, height=600)

Your results may vary. But you're likely to see latitude and longitude (i.e., location) on top, along with population size and median income, which are stronger predictors of housing prices than the other features in the dataset.

In SageMaker Studio, you can also navigate to SageMaker resources tab, click on Experiments and trials, and find your Autopilot experiment. You can double-click on the experiment name to list all trials, and from there you can double-click on a specific trial to see its details, including charts and metrics.

---
## Evaluate Model Using Test Dataset<a name="Evaluate"></a>

To evaluate the model on previously unseen data, we will test it against the test dataset we prepared earlier. For that, we don't necessarily need to deploy the model to an endpoint, we can simply run a batch transform job to get predictions for our unlabeled test dataset.

### Set up Transform Job

In [20]:
from sagemaker import AutoML

automl = AutoML.attach(auto_ml_job_name=auto_ml_job_name)

s3_transform_output_path = "s3://{}/{}/inference-results/".format(s3_bucket, prefix)

model_name = "{0}-model".format(best_candidate_name)

model = automl.create_model(
    name=model_name,
    candidate=best_candidate,
)

output_path = s3_transform_output_path + best_candidate_name + "/"

transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m5.xlarge",
    assemble_with="Line",
    strategy="SingleRecord",
    output_path=output_path,
    env={"SAGEMAKER_MODEL_SERVER_TIMEOUT": "100", "SAGEMAKER_MODEL_SERVER_WORKERS": "1"},
)

### Launch Transform Job

In [21]:
transformer.transform(
    data=test_data_s3_path,
    split_type="Line",
    content_type="text/csv",
    wait=False,
    model_client_config={"InvocationsTimeoutInSeconds": 80, "InvocationsMaxRetries": 1},
)

print("Starting transform job {}".format(transformer._current_job_name))

Starting transform job automl-housing-20220812-16-18wm1-010-35-2022-08-12-16-59-51-180


#### Track Transform Job Status

In [22]:
## Wait for jobs to finish
pending_complete = True
job_name = transformer._current_job_name

while pending_complete:
    pending_complete = False

    description = sm.describe_transform_job(TransformJobName=job_name)
    if description["TransformJobStatus"] not in ["Failed", "Completed"]:
        pending_complete = True

    print("{} transform job is running.".format(job_name))
    time.sleep(60)

print("\nCompleted.")

automl-housing-20220812-16-18wm1-010-35-2022-08-12-16-59-51-180 transform job is running.
automl-housing-20220812-16-18wm1-010-35-2022-08-12-16-59-51-180 transform job is running.
automl-housing-20220812-16-18wm1-010-35-2022-08-12-16-59-51-180 transform job is running.
automl-housing-20220812-16-18wm1-010-35-2022-08-12-16-59-51-180 transform job is running.
automl-housing-20220812-16-18wm1-010-35-2022-08-12-16-59-51-180 transform job is running.
automl-housing-20220812-16-18wm1-010-35-2022-08-12-16-59-51-180 transform job is running.
automl-housing-20220812-16-18wm1-010-35-2022-08-12-16-59-51-180 transform job is running.

Completed.


### Evaluate the Inference Results

The transform job will have now generated a CSV file with inference results for the test dataset. We will use those results and compare them with the real test labels to see how the model performs compared to real data.

In [23]:
def get_csv_from_s3(s3uri, file_name):
    parsed_url = urlparse(s3uri)
    bucket_name = parsed_url.netloc
    prefix = parsed_url.path[1:].strip("/")
    s3 = boto3.resource("s3")
    obj = s3.Object(bucket_name, "{}/{}".format(prefix, file_name))
    return obj.get()["Body"].read().decode("utf-8")


job_status = sm.describe_transform_job(TransformJobName=job_name)["TransformJobStatus"]

if job_status == "Completed":
    pred_csv = get_csv_from_s3(transformer.output_path, "{}.out".format(test_file))
    predictions = pd.read_csv(io.StringIO(pred_csv), header=None)

In [24]:
from sklearn.metrics import mean_squared_error
from math import sqrt

labels_df = test_data[target]
mse = mean_squared_error(labels_df, predictions)
rmse = sqrt(mse)

print("MSE: {0}\nRMSE: {1}".format(mse, rmse))

MSE: 1929486293.4474235
RMSE: 43925.91824250716


---
## Cleanup

The Autopilot job creates many underlying artifacts such as dataset splits, preprocessing scripts, or preprocessed data, etc. This code, when un-commented, deletes them. 

In [25]:
s3 = boto3.resource("s3")
s3_bucket = s3.Bucket(bucket)

print(s3_bucket)
job_outputs_prefix = "{}/output/{}".format(prefix, auto_ml_job_name)
print(job_outputs_prefix)

# Delete S3 objects
s3_bucket.objects.filter(Prefix=job_outputs_prefix).delete()

s3.Bucket(name='sagemaker-us-west-2-060090418572')
sagemaker/DEMO-autopilot-housing/output/automl-housing-20220812-16-18


[{'ResponseMetadata': {'RequestId': 'AC1E6ETA34KTBTDW',
   'HostId': 'Z+/87x9GySxyH0D4L3jE6MukvrjB859vINFKWsIySiWUTr5fDbYm68/SWADOCC/6Cd0u4u8kfluwvwuVtdhlXg==',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'Z+/87x9GySxyH0D4L3jE6MukvrjB859vINFKWsIySiWUTr5fDbYm68/SWADOCC/6Cd0u4u8kfluwvwuVtdhlXg==',
    'x-amz-request-id': 'AC1E6ETA34KTBTDW',
    'date': 'Fri, 12 Aug 2022 17:06:54 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-autopilot-housing/output/automl-housing-20220812-16-18/transformed-data/dpp5/rpb/train/chunk_37.csv.out'},
   {'Key': 'sagemaker/DEMO-autopilot-housing/output/automl-housing-20220812-16-18/transformed-data/dpp2/rpb/validation/chunk_18.csv.out'},
   {'Key': 'sagemaker/DEMO-autopilot-housing/output/automl-housing-20220812-16-18/transformed-data/dpp1/rpb/validation/chunk_19.csv.out'},
   {'Key': 'sagemaker/D

We then delete all the experiment and model resources created by the Autopilot experiment.

In [26]:
def cleanup_experiment_resources(experiment_name):
    trials = sm.list_trials(ExperimentName=experiment_name)["TrialSummaries"]
    print("TrialNames:")
    for trial in trials:
        trial_name = trial["TrialName"]
        print(f"\n{trial_name}")

        components_in_trial = sm.list_trial_components(TrialName=trial_name)
        print("\tTrialComponentNames:")
        for component in components_in_trial["TrialComponentSummaries"]:
            component_name = component["TrialComponentName"]
            print(f"\t{component_name}")
            sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
            try:
                # comment out to keep trial components
                sm.delete_trial_component(TrialComponentName=component_name)
            except:
                # component is associated with another trial
                continue
            # to prevent throttling
            time.sleep(5)
        sm.delete_trial(TrialName=trial_name)
    sm.delete_experiment(ExperimentName=experiment_name)
    print(f"\nExperiment {experiment_name} deleted")


def cleanup_autopilot_models(autopilot_job_name):
    print("{0}:\n".format(autopilot_job_name))
    response = sm.list_models(NameContains=autopilot_job_name)

    for model in response["Models"]:
        model_name = model["ModelName"]
        print(f"\t{model_name}")
        sm.delete_model(ModelName=model_name)
        # to prevent throttling
        time.sleep(3)

In [27]:
cleanup_experiment_resources("{0}-aws-auto-ml-job".format(auto_ml_job_name))

TrialNames:

automl-housing-20220812-16-18wm1-001-119eff2e-aws-trial
	TrialComponentNames:
	automl-housing-20220812-16-18wm1-001-119eff2e-aws-training-job
	automl-housing-20220812-16-18-dpp7-rpb-1-273f332cd8d44c708986cf-aws-transform-job
	automl-housing-20220812-16-18-dpp7-1-03e643c2246b459ea431e7c827-aws-training-job
	automl-housing-20220812-16-18-db-1-b51a80c4215742a88aaec179b3f7-aws-processing-job

automl-housing-20220812-16-18wm1-006-e5194d10-aws-trial
	TrialComponentNames:
	automl-housing-20220812-16-18wm1-006-e5194d10-aws-training-job
	automl-housing-20220812-16-18-dpp3-rpb-1-d8dfdd9921e342d0b42ed5-aws-transform-job
	automl-housing-20220812-16-18-dpp3-1-0f7b3cb6279146899f4ba35280-aws-training-job
	automl-housing-20220812-16-18-db-1-b51a80c4215742a88aaec179b3f7-aws-processing-job

automl-housing-20220812-16-18wm1-002-9e9f5dcd-aws-trial
	TrialComponentNames:
	automl-housing-20220812-16-18wm1-002-9e9f5dcd-aws-training-job
	automl-housing-20220812-16-18-dpp9-csv-1-593dd40fdd114d9ba33

In [28]:
cleanup_autopilot_models(auto_ml_job_name)

automl-housing-20220812-16-18:

	automl-housing-20220812-16-18wm1-010-352d0c11-model
	automl-housing-20220812-16-18-dpp5-model-11e7322eb0664e9e9e4132
	automl-housing-20220812-16-18-dpp4-model-ff1ed63219114c32a15c61
	automl-housing-20220812-16-18-dpp6-model-30a41b7c483f4a65b61059
	automl-housing-20220812-16-18-dpp7-model-2d26e4b145a346afaff404
	automl-housing-20220812-16-18-dpp2-model-8dd9d68503fa4b849ad601
	automl-housing-20220812-16-18-dpp3-model-c1f3831dffda48cfbae378
	automl-housing-20220812-16-18-dpp0-model-e7a09b43cda746f58a5c0d
	automl-housing-20220812-16-18-dpp8-model-8aaeba52380547b9853650
	automl-housing-20220812-16-18-dpp1-model-2805f517e9b845ab9b85d2


Finally, the following code, when uncommented, will delete the local files used in this demo.

In [29]:
import shutil
import glob
import os


def delete_local_files():
    base_path = ""
    dir_list = glob.iglob(os.path.join(base_path, "{0}*".format(auto_ml_job_name)))

    for path in dir_list:
        if os.path.isdir(path):
            shutil.rmtree(path)

    if os.path.exists("CaliforniaHousing"):
        shutil.rmtree("CaliforniaHousing")

    if os.path.exists("cal_housing.tgz"):
        os.remove("cal_housing.tgz")

    if os.path.exists("SageMakerAutopilotCandidateDefinitionNotebook.ipynb"):
        os.remove("SageMakerAutopilotCandidateDefinitionNotebook.ipynb")

    if os.path.exists("SageMakerAutopilotDataExplorationNotebook.ipynb"):
        os.remove("SageMakerAutopilotDataExplorationNotebook.ipynb")

    if os.path.exists("test_data_no_target.csv"):
        os.remove("test_data_no_target.csv")

    if os.path.exists("test_data.csv"):
        os.remove("test_data.csv")

    if os.path.exists("train_data.csv"):
        os.remove("train_data.csv")


## UNCOMMENT TO CLEAN UP LOCAL FILES
# delete_local_files()

**Note: If you enabled automatic endpoint creation, you will need to delete the endpoint manually.**